In [46]:
import pandas as pd
from tqdm import tqdm

In [47]:
from difflib import SequenceMatcher

def similar(a, b):
    return SequenceMatcher(None, a, b).ratio()

def jaccard_similarity(text1, text2):
    text1_set = set(text1)
    text2_set = set(text2)
    intersection = text1_set.intersection(text2_set)
    union = text1_set.union(text2_set)
    return len(intersection) / len(union)

In [48]:
# TODO 전처리한 title의 similarity를 계산합니다. -> 전처리 title을 기준으로 item df를 sort합니다 -> 게슈탈트 패턴 매칭 알고리즘을 사용해서 계산합니다. -> 이는 재사용을 위해 pickle 객체로 저장합니다.
# TODO item df를 전처리한 title 기준으로 sort 후에, 마찬가지로 전처리한 title 기준으로 정렬된 pickle 객체를 불러와서 merge합니다.
# TODO 추천에 활용하지 않을 category 목록들을 필터링합니다.
# TODO 필터링 이후, similarity에 대한 BFS 알고리즘을 적용합니다 -> 이전에 구한 similarity가 전체 item을 기준으로 구해졌기 때문에, 예외 처리를 해줘야 합니다.
# TODO BFS 알고리즘을 통해 찾아낸 clustering group을 만들어주고, "ETC"나 "etc"로 group된 item들은 -1로 변환합니다.
# TODO house_interaction에 클러스터 결과를 merge해줍니다. 이때, 클러스터 그룹이 -1이거나 house_interaction에만 포함된 item들은 cluster_id.max() + 1을 더해줍니다.
# TODO model을 학습해서 학습한 결과를 backend에 반영합니다.
# TODO cluster-major-item list를 생성합니다.

In [49]:
item = pd.read_csv(f"title_preprocessed_item_93886.csv")

In [50]:
item.sort_values("preprocessed_title", inplace=True)

In [51]:
len(item)

147780

In [52]:
item.preprocessed_title.fillna("etc", inplace=True)
item.preprocessed_title = item.preprocessed_title.str.lower()

In [55]:
window = 100
threshold = 0.7

sim_list = []

for i, (id, t) in tqdm(enumerate(zip(item.item[:len(item) - window + 1], item.preprocessed_title[:len(item) - window + 1]))):
    word = t
    tmp = []
    for j, (id_, t_) in enumerate(zip(item.item[i + 1:i + window], item.preprocessed_title[i + 1:i + window])):
        compare = t_
        similarity = similar(word, compare)
        if similarity >= threshold:
            tmp.append([word, compare, id_, similarity])
    sim_list.append(tmp)

147681it [22:37, 108.75it/s]


In [70]:
start = len(sim_list)
end = len(item)

sim_list2 = []

for i, (id, t) in tqdm(enumerate(zip(item.item[start:end], item.preprocessed_title[start:end]))):
    word = t
    tmp = []
    for j, (id_, t_) in enumerate(zip(item.item[i + start + 1:end], item.preprocessed_title[i + start + 1:end])):
        compare = t_
        similarity = similar(word, compare)
        if similarity >= threshold:
            tmp.append([word, compare, id_, similarity])
    sim_list2.append(tmp)

99it [00:00, 362.34it/s]


In [72]:
data = sim_list[:]
for d in sim_list2:
    data.append(d)

In [74]:
len(data)

147780

In [75]:
import pickle
from time import time
from difflib import SequenceMatcher
import pickle

# now = str(round(time()))[5:]
with open(f"similarity_{now}.pickle","wb") as pkl:
    pickle.dump(data, pkl)

In [76]:
with open(f"similarity_{now}.pickle", "rb") as pkl:
    d = pickle.load(pkl)

In [77]:
len(d)

147780